In [ ]:
import numpy as np

##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from matplotlib import pyplot as plt

%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

global_fig_width = 8
global_fig_height = global_fig_width / 1.61803399
font_size = 12

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

# Likelihood

- Recall the likelihood appears in Bayes' Theorem 

$$P(\theta | data) = \frac{\color{red}{{P(data|\theta)}} P(\theta)}{P(data)}$$

- Remember not a probability distribution because $\theta$ varies
- Most important choice, derived from the statistical model of the underlying process
- Encapsulates many subjective judgements about analysis.

# Equivalence relation

- A notation often seen in the literature is

$$\mathcal{L}(\theta | data) = P(data | \theta)$$

Therefore, a likelihood of $\theta$ for a particular data sample is equivalent to the probability of that data sample for that value of $\theta$. We call the above an *equivalence relation*

# Example:  frequency of lift malfunctioning


- Imagine we want to create a model for the frequency a lift (elevator) breaks down in a given year, $X$.
- Assume a range of unpredictable and uncorrelated factors (temperature, lift usage, etc.) affect the functioning of the lift.
- therefore, $X ∼ \text{Poisson}(\theta)$, where $\theta$ is the mean number of times the lift breaks in one year.
- we don’t a priori know the true value of $\theta$, our model defines collection of probability models; one for each value of $\theta$.
- We call this collection of models the Likelihood.

![](fig/miscellaneous-elevator_repairman-elevators-lifts-repairs-repairmen-mban2852_low.jpg)


In [ ]:
import math
from scipy.stats import poisson
def lift_likelihood(theta, data):
    x = np.arange(0, 20)
    plt.plot(x, poisson.pmf(x, theta), 'bo', ms=8, label='poisson pmf')
    plt.vlines(x, 0, poisson.pmf(x, theta), colors='b', lw=5, alpha=0.5)
    print(data)
    if data is not None:
        plt.vlines([data], 0, poisson.pmf(data, theta), colors='r', lw=5, alpha=0.9)
    plt.ylim(0,0.25)
    plt.ylabel(r'$P(X | \theta=%s)$'%str(theta))
    plt.xlabel('X')
    plt.show()
def lift_likelihood_no_data(theta):
    lift_likelihood(theta,None)
def lift_likelihood_five_data(theta):
    lift_likelihood(theta,5)
    
def lift_likelihood_w_theta(k):
    theta = np.linspace(0, 20, 100)
    plt.plot(theta, np.exp(-theta)*theta**k/(math.factorial(k)))
    plt.ylabel(r'$P(k=%s | \theta)$'%str(k))
    plt.xlabel(r'mean number of breakdowns $\theta$')
    plt.show()

In [ ]:
# Display our collection of models
widget = FloatSlider(value=5.0, min=0.0, max=15.0, step=1.0, continuous_update=False)
interact(lift_likelihood_no_data, theta=widget, continuous_update=False);

To calculate the likelihood:
- fix the data (say number of breakdowns is measured at 5)
- find the corresponding probability for each of the models

In [ ]:
interact(lift_likelihood_five_data, theta=widget, continuous_update=False);

Using the equation for a Poisson distribution, and $k=5$ as the number of breakdowns, the likelihood function is:
    
$$\mathcal{L}(\theta | k) = P(k | \theta) = \exp(-\theta)\frac{\theta^k}{k!}$$

In [ ]:
lift_likelihood_w_theta(5)

# Example: ODE-based model

- Models can often take the form of differential equations evolving in time
- Likelihood is often straightforward to derive and relativly cheap to calculate due to *independent* (in time) measurement noise 
- Lets consider the case of an ordinary differential equation (ODE) model, the reversible reaction model in the previous lecture:

$$\dot{y}(t) = k_1 (1 - y) - k_2 y,$$

where $k_1$ represents a forward reaction rate, $k_2$ is the backward reaction rate, and $y$ represents the concentration of a chemical solute.

- In an experiment, we take $N$ measurements of the system $z_i$, with $i = 0...N-1$, that are modelled with independent Gaussian measurement noise:

$$z_i \sim y(t) + N(0, \sigma)$$

- Assuming that $\sigma$ is unknown, we now have **three** model parameters: $\boldsymbol{\theta} = [k_1, k_2, \sigma]$, rather than one in the previous lift example


Lets look at a random realisations of this experiment, with $N=50$, $k_1=5$, $k_2=3$, and $\sigma=0.02$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def r(y, t, p):
    k1 = p[0] 
    k2 = p[1] 
    dydt = k1 * (1 - y) - k2 * y
    return dydt

def random(p):
    y0 = 0.1 
    times = np.linspace(0, 1, 50)
    values = odeint(r, y0, times, (p,))
    values += np.random.normal(0, 0.02, values.shape)
    plt.ylabel('Concentration')
    plt.scatter(times, values)
    plt.show()

In [ ]:
random([5,3])

- As before, this model describes an infinite family of probability distributions governed by the three parameters $\boldsymbol{\theta} = [k_1, k_2, \sigma]$
- However, now we have $N$ outputs due to the $N$ time samples that were measured, therefore a probabilty distribution around each time point $t_i$

In [ ]:

from scipy.stats import norm
def family_of_models(k1, k2, sigma):
    y0 = 0.1 
    times = np.linspace(0, 1, 50)
    index = int(len(times)/2)
    values = odeint(r, y0, times, ([k1, k2],)).reshape(-1)
    global_fig_width = 8
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8/1.618))
    ax1.set_ylabel('Concentration')    
    ax1.set_xlabel(r'$t$')
    ax1.plot(times, values)
    ax1.fill_between(times, values-sigma, values+sigma, alpha=0.5)
    ax1.axvline(times[index],color='r')
    
    concentrations = np.linspace(0, 1, 100)
    ax2.plot(concentrations, norm.pdf(concentrations, values[index], sigma), color='r')
    ax2.set_xlabel('concentration')
    ax2.set_ylabel(r'$P(y(t_i) | \theta)$')
    plt.show()

k1_widget = FloatSlider(value=5.0, min=0.0, max=15.0, step=1.0, continuous_update=False)
k2_widget = FloatSlider(value=3.0, min=0.0, max=15.0, step=1.0, continuous_update=False)
sigma_widget = FloatSlider(value=0.1, min=0.0, max=0.2, step=0.01, continuous_update=False)

In [ ]:
# Display our collection of models
interact(family_of_models, k1=k1_widget, k2=k2_widget, sigma=sigma_widget, continuous_update=False);

# Likelihood of ODE model

- we assume that the errors at each time point are independent, therefore the conditional probability density of observing the whole experimental trace from time sample 1 to time sample N is simply the product of the probability density functions at each time point

$$\mathcal{L}(\theta \vert \mathbf{y}) = \prod_{i=1}^{N} P(z_i | \boldsymbol{\theta}).$$

- With our further assumption that the experimental noise is also normally distributed with a mean of zero and variance of $\sigma^2$ , the likelihood can be expressed as

\begin{equation}
L(\boldsymbol{\theta} \vert \mathbf{z}) =  \prod_{i=1}^{N} \mathcal{N}(z_i \vert y(t, \boldsymbol{\theta}),\sigma^2) = \prod_{i=0}^{N} \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left(-\frac{\left(z_i -y(t_i, \boldsymbol{\theta})\right)^2}{2\sigma^2}\right),
\end{equation}

- Normally work with the log-likelihood:

\begin{align}
l(\boldsymbol{\theta} \vert \mathbf{z}) &= \log(L(\boldsymbol{\theta} \vert \mathbf{z})) \nonumber \\
&= -\frac{N}{2}\log(2\pi\sigma^2)-\frac{1}{2\sigma^2}\sum_{i=1}^N (z_i - y(t_i, \boldsymbol{\theta}))^2 \nonumber \\
&= -\frac{N}{2}\log(2\pi) - N\log(\sigma) - \frac{1}{2\sigma^2}\sum_{i=1}^N (z_i - y (t_i, \boldsymbol{\theta}))^2
\end{align}

- Notice the similarities with classical sum of squares error function (for fixed $\sigma$ anyway)!

# Maximum Likelihood Estimation

- In the previous lecture we worked out the posterior *distribution* a given parameter, given the available data
- Often only interested in the most *likely* parameter value
- Can use maximum likelihood estimation, simply find the value of \theta that maximises the likelihood
- a **frequentist** approach, uses a likelihood function (not a valid probability distribution)

$$\theta_{mle} = \text{arg max}_{\theta \in \Omega} P(data|\theta)$$

# Coin example

![](fig/coin.jpeg)

- Back to the coin example. We perform an experiment of $n$ flips, and it lands heads up $h$ times

- Likelihood is:

$$L(\theta \vert h\times H) = \theta^h (1-\theta)^{n-h}$$

# Finding the parameters that maximise the Likelihood

- For tractable likelihood functions, we can simply find the derivative of the likelihood and set to zero:
    
$$\frac{\partial L(\theta \vert h\times H)}{\partial \theta} = \theta^{h - 1} (h - n t) (-t + 1)^{-h + n - 1}$$

- Set $\frac{\partial L(\theta \vert h\times H)}{\partial \theta} = 0$ to find the MLE estimator for $\theta$

$$\theta_{mle} = \frac{h}{n}$$


# Non-linear optimisation

- Often the analytical approach is not feasible, can then turn to one of many non-linear optimisation algorithms
- Derivative-free Direct methods (aka Search methods)
  - Brute-force exploration (look at the landscape and pick the lowest point)
  - Random search (e.g. Simulated annealing) 
  - Coordinate search (aka Coordinate descent, aka Compass search)
  - Search & poll / Pattern search
  - Simplex methods (e.g. Nelder-Mead)
  - Tabu search
  - Dividing rectangles algorithm (DIRECT)
  - Powell's conjugate direction method
- Multi-start methods
  - Multi-level single-linkage (MLSL)
  - Basin-hopping
- Derivative-free evolutionary methods and metaheuristics
  - Genetic algorithms (GA)
  - Differential evolution
  - Evolution strategies
  - Controlled random search (CRS)
  - Swarm algorithms
  - Metaheuristics
- Bayesian optimistation
- Gradient-estimating methods
  - Finite difference methods
  - Simplex gradient methods / Implicit filtering
  - Natural evolution strategies (NES)
- Surrogate-model methods
  - Trust-region methods
  - Data-based Online Nonlinear Extremumseeker (DONE)
- Methods requiring the 1st-order gradient
  - Root finding methods (e.g. Newton's method, BFGS)
  - Gradient descent (aka Steepest descent)
  - Stochastic gradient descent
  - Continuation

# Some useful non-linear optimisation packages:

- [NLopt](https://nlopt.readthedocs.io)
- [Pagmo/Pygmo](http://esa.github.io/pygmo/)
- [Scipy](https://docs.scipy.org/doc/scipy/reference/optimize.html)
- [Pints](https://pints.readthedocs.io/en/latest/)

# Example using PINTS

- We will use a popular model of population growth, the logistic equation:

    $$ \frac{df(t)}{dt} = r f(t) \frac{k - f(t)}{k}$$
    $$f(t) = \frac{k}{1+(k/p_0 - 1) \exp(-r t)}$$
    
- Two parameters, the carrying capacity $k$, and the rate of growth $r$
- We will assume an unknown measurement noise $\sigma$, which gives the parameter set $\boldsymbol{\theta} = [r, k, \sigma]$
- The `pints.GaussianLogLikelihood` in PINTS implements the independent Gaussian noise log-likelihood derived earlier

\begin{align}
l(\boldsymbol{\theta} \vert \mathbf{z}) &= \log(L(\boldsymbol{\theta} \vert \mathbf{z})) \nonumber \\
&= -\frac{N}{2}\log(2\pi) - N\log(\sigma) - \frac{1}{2\sigma^2}\sum_{i=1}^N (z_i - y (t_i, \boldsymbol{\theta}))^2
\end{align}

In [ ]:
import pints
import pints.toy
import matplotlib.pyplot as plt
import numpy as np

p0 = 1  # initial population; initial value
model = pints.toy.LogisticModel(p0)

# Define the 'true' parameters
true_parameters = [0.1, 50, 5]

# Run a simulation to get test data
times = np.linspace(0, 100, 100)
values = model.simulate(true_parameters[:-1], times)

# Add some noise
values += np.random.normal(0, true_parameters[-1], values.shape)


In [ ]:
# Show the test data
plt.figure()
plt.xlabel('Time')
plt.ylabel(r'Population $y(t)$')
plt.scatter(times, values, label='data')
plt.plot(times, model.simulate(true_parameters[:-1], times), color='r', lw=3, label='true parameters')
plt.legend()
plt.show()

In [ ]:
# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create the log-likelihood function
log_likelihood = pints.GaussianLogLikelihood(problem)

# Select some boundaries
boundaries = pints.RectangularBoundaries([0, 0, 0], [100, 100, 100])

# Select a starting point
x0 = [50, 50, 50]

# Perform an optimization using XNES. 
found_parameters, found_value = pints.optimise(log_likelihood, x0, boundaries=boundaries, method=pints.XNES)
print('log_likelihood at true solution:')
print(log_likelihood(true_parameters))

In [ ]:
# Show the results
plt.figure()
plt.xlabel('Time')
plt.ylabel(r'Population $y(t)$')
found_mean = model.simulate(found_parameters[:-1], times)
plt.fill_between(times, found_mean - found_parameters[-1], found_mean + found_parameters[-1],
                 color='gray', alpha=0.3)
plt.plot(times, found_mean, color='r', label='found parameters')
plt.scatter(times, values, alpha=0.5, label='data')
plt.legend()
plt.show()

# Maximum a posteriori (MAP) estimation

- Rather than a likelihood, can also maximise the unnormalised posterior:

$$P(\theta | data) \sim P(data|\theta) P(\theta)$$

$$\theta_{map} = \text{arg max}_{\theta \in \Omega}  P(data|\theta) P(\theta)$$

- MLE is a particular case of MAP, using a Uniform prior (just multiplies the likelihood by a constant)
- This method is a useful method of incorporporating domain knowledge on the parameters
- Related to regularisation in non-linear and linear optimisation


# Coin example with a Gaussian prior

![](fig/coin.jpeg)

- Reasonable assumption is that the coin is likely to be fair, lets use a Gaussian prior $\mathcal{N}(0.5, \sigma)$

- Maximum a posteriori loss function $L(\theta)$ is therefore:

\begin{align*}
L(\theta) &= P(data|\theta) P(\theta) \\ 
&= \theta^h (1-\theta)^{n-h} \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(\theta - 0.5)^2}{2 \sigma^2}}
\end{align*}

In [ ]:
def show_map_coin(h, sigma):
    n = 10
    theta = np.linspace(0,1,100)
    L = theta**h * (1-theta)**(n-h) * np.exp(-(theta - 0.5)**2/(2*sigma**2)) / np.sqrt(2*np.pi*sigma**2)
    plt.plot(theta,L, label='loss function')
    plt.xlabel(r'Probability of landing heads $\theta$')
    plt.ylabel(r'MAP loss function $L(\theta)$')
    max_i = np.argmax(L)
    plt.scatter([theta[max_i]],[L[max_i]], color='r', label='MAP estimate', s=100)
    plt.legend()
    plt.show()
    
h_widget = FloatSlider(value=8.0, min=0.0, max=10.0, step=1.0, continuous_update=False)
sigma_widget = FloatSlider(value=0.3, min=0.0, max=0.3, step=0.01, continuous_update=False)

In [ ]:
interact(show_map_coin, h=h_widget, sigma=sigma_widget, continuous_update=False);

# Logistic growth example

Recall the logistic equation:
    
$$f(t) = \frac{k}{1+(k/p_0 - 1) \exp(-r t)}$$

- Anyone familiar with this equation could estimate a value of the carrying capacity $k$ from a plot
- Would be reasonable to therefore use a Gaussian Prior for $k$
  

In [ ]:
def show_logistic_estimate():
    plt.xlabel('Time')
    plt.ylabel(r'Population $y(t)$')
    plt.scatter(times, values, label='data')
    plt.plot([0, 100], [50, 50], c='k', ls='--', lw=3, label='estimate for $k$')
    plt.legend()
    plt.show()

In [ ]:
print(values)
show_logistic_estimate()

## Finding MAP estimator in PINTS

In [ ]:
# Create the log-likelihood function (using the problem defined earlier)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over r
log_prior_r = pints.UniformLogPrior([0],[100])

# Create a gaussian prior over k
log_prior_k = pints.GaussianLogPrior(50,10)

# Create a uniform prior over sigma
log_prior_sigma = pints.UniformLogPrior([0],[100])

# Create a composed prior
log_prior = pints.ComposedLogPrior(log_prior_r, log_prior_k, log_prior_sigma)

In [ ]:
# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Select some boundaries
boundaries = pints.RectangularBoundaries([0, 0, 0], [100, 100, 100])

# Select a starting point
x0 = [50, 50, 50]

# Perform an optimization using Particle Swarm Optimisation (PSO). 
found_parameters, found_value = pints.optimise(log_likelihood, x0, boundaries=boundaries, method=pints.PSO)
print('posterior log-likelihood at true solution:')
print(log_posterior(true_parameters))

In [ ]:
# Show the results
plt.figure()
plt.xlabel('Time')
plt.ylabel(r'Population $y(t)$')
found_mean = model.simulate(found_parameters[:-1], times)
plt.fill_between(times, found_mean - found_parameters[-1], found_mean + found_parameters[-1],
                 color='gray', alpha=0.2)
plt.plot(times, found_mean)
plt.plot(times, values)
plt.show()

# Electrochemistry example - POMS

- three unresolved two-electron surface-confined polyoxometalate reduction processes by AC voltammetry

![](fig/pom.svg)
**(left)** Molecular structure of $[\text{PMo}_{12}\text{O}_{40}]^{3-}$    **(right)** Experimental AC voltammetry trace

- The sequence of six electron transfer steps are modelled by the following quasi-reversible reactions

\begin{align}
    A + e^- \underset{k^1_{ox}(t)}{\overset{k^1_{red}(t)}{\rightleftarrows}} B,
    \\
    B + e^- \underset{k^2_{ox}(t)}{\overset{k^2_{red}(t)}{\rightleftarrows}} C,
    \\
    C + e^- \underset{k^3_{ox}(t)}{\overset{k^3_{red}(t)}{\rightleftarrows}} D,
    \\
    D + e^- \underset{k^4_{ox}(t)}{\overset{k^4_{red}(t)}{\rightleftarrows}} E,
    \\
    E + e^- \underset{k^5_{ox}(t)}{\overset{k^5_{red}(t)}{\rightleftarrows}} F,
    \\
    F + e^-  \underset{k^6_{ox}(t)}{\overset{k^6_{red}(t)}{\rightleftarrows}} G,
\end{align}

where the forward $k_{red}$ and backwards $k_{ox}$ reaction rates are
given by
the Butler-Volmer
relationships

\begin{align}\label{eq:rate1}
    k^i_{red}(t) &= k^0_i \exp\left(-\frac{\alpha_i F}{RT} [E_r(t) - E^0_i]
    \right), \\
    k^i_{ox}(t) &= k^0_i \exp\left((1-\alpha_i)\frac{F}{RT} [E_r(t) - E^0_i]
\right).  \label{eq:rate2}
\end{align}

- This can be modelled by an ordinary differential equation containing 17 parameters to be estimated

$$
\mathbf{p} =
(E^0_1,E^0_2,E^0_3,E^0_4,E^0_5,E^0_6,k^0_1,k^0_2,k^0_3,k^0_4,k^0_5,k^0_6,
         \alpha_1,
         \alpha_2,
         R_u,
         C_{dl},
         \Gamma).
$$

- The effect of the $E^0_i$ parameters on the simulated current is highly non-linear.
- In such a high dimensional space all non-linear optimisers we tried failed to find the global minimum
- But approximate values of $E^0_i$ can be easily read off the experimental current trace.... **solution:** put a Gaussian prior on all $E^0_i$ parameters

**standard deviation** of the Gaussian prior (i.e. confidence of the estimation of $E^0_i$), required to be $<= 0.1$ V for **reliable parameter estimation**

![](fig/quasireversible.svg)

# Revisiting the independent noise assumption

![](fig/danger.png)

- Assuming independent, Gaussian measurement noise results in the following log-likelihood:

- Independent noise is easy to use, and often makes intuative sense.
- **Check that this assumption is valid.**